# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [26]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [170]:
# 1) Consultando os dados da API e filtrando apenas para o estado do RJ e tipo "Município"
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
params = {"uf": "RJ", "tipo_interessado": "Município"}

response = requests.get(url, params=params)
dados = response.json()

df = pd.DataFrame(dados["items"])
df.head()

In [171]:
# 2) Três status mais frequentes
df['status'].value_counts().head(3)

In [172]:
# 3) Seu códigExtraindo o ano da coluna data_status
df["ano_status"] = df["data_status"].str.slice(0, 4)
df["ano_status"].value_counts()o aqui


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [5]:
#1) Função para consultar API por UF e tipo de interessado
import requests
import pandas as pd

def consultar_pvl(uf=None, tipo_interessado=None):
    """
    Consulta a API do Tesouro Nacional para obter dados de PVL.

    Args:
        uf (str, opcional): Sigla da Unidade Federativa para filtrar. Defaults to None.
        tipo_interessado (str, opcional): Tipo de interessado ('Estado' ou 'Município') para filtrar. Defaults to None.

    Returns:
        pandas.DataFrame: DataFrame com os dados da consulta.
    """
    url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    params = {}
    if uf:
        params['uf'] = uf
    if tipo_interessado:
        params['tipo_interessado'] = tipo_interessado

    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    return pd.DataFrame(data['items'])

In [174]:
# 2) # Consultar MG com tipo interessado "Estado"
df_mg_estados = consultar_pvl(uf="MG", tipo_interessado="Estado")
arquivados_mg = df_mg_estados[df_mg_estados['status'] == 'Arquivado por decurso de prazo']
quantidade_arquivados_mg = len(arquivados_mg)
print(f"O Estado de Minas Gerais possui {quantidade_arquivados_mg} solicitações arquivadas por decurso de prazo.")

In [175]:
# 3) Consultar BA com tipo interessado "Município"
df_ba_municipios = consultar_pvl(uf="BA", tipo_interessado="Município")
deferidas_ba = df_ba_municipios[df_ba_municipios['status'] == 'Deferido']
contagem_deferimentos = deferidas_ba['municipio'].value_counts()
municipio_mais_deferimentos = contagem_deferimentos.index[0]
mais_solicitacoes = contagem_deferimentos.iloc[0]
print(f"O município da Bahia com mais solicitações deferidas é '{municipio_mais_deferimentos}' com {mais_solicitacoes} deferimentos.")

In [6]:
# 4) Consultar BA com tipo interessado "Estado"
df_ba_estados = consultar_pvl(uf="BA", tipo_interessado="Estado")
df_ba_estados.to_csv("solicitacoes_bahia_estados.csv", index=False, encoding='utf-8')
print("Os dados de solicitações do estado da Bahia foram salvos em 'solicitacoes_bahia_estados.csv'.")

Os dados de solicitações do estado da Bahia foram salvos em 'solicitacoes_bahia_estados.csv'.
